In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType, StringType
from pyspark import SparkContext
from pyspark.sql import SQLContext
from datetime import datetime

glueContext = GlueContext(SparkContext.getOrCreate())
s3_bucket = "s3://glueworkshop-ray-us-east-2"
output_path = s3_bucket + "/output/lab4/notebook/"
job_time_string = datetime.now().strftime("%Y%m%d%H%M%S")
s3_target = output_path + job_time_string

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1623861101193_0007,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
country_lookup_frame = glueContext.create_dynamic_frame.from_options(
                            format_options = {"withHeader":True, "separator":",", "quoteChar":"\""},
                            connection_type = "s3", 
                            format = "csv", 
                            connection_options = {"paths": [s3_bucket + "/input/lab4/country_lookup/"], "recurse":True}, 
                            transformation_ctx = "coutry_lookup_frame")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def processBatch(data_frame, batchId):
    if (data_frame.count() > 0):
        dynamic_frame = DynamicFrame.fromDF(data_frame, glueContext, "from_data_frame")
        apply_mapping = ApplyMapping.apply(frame=dynamic_frame, mappings=[
            ("uuid", "string", "uuid", "string"),
            ("country", "string", "country", "string"),
            ("itemtype", "string", "itemtype", "string"),
            ("saleschannel", "string", "saleschannel", "string"),
            ("orderpriority", "string", "orderpriority", "string"),
            ("orderdate", "string", "orderdate", "string"),
            ("region", "string", "region", "string"),
            ("shipdate", "string", "shipdate", "string"),
            ("unitssold", "string", "unitssold", "string"),
            ("unitprice", "string", "unitprice", "string"),
            ("unitcost", "string", "unitcost", "string"),
            ("totalrevenue", "string", "totalrevenue", "string"),
            ("totalcost", "string", "totalcost", "string"),
            ("totalprofit", "string", "totalprofit", "string")],
                                           transformation_ctx="apply_mapping")

        final_frame = Join.apply(apply_mapping, country_lookup_frame, 'country', 'CountryName').drop_fields(
            ['CountryName', 'country', 'unitprice', 'unitcost', 'totalrevenue', 'totalcost', 'total profit'])

        s3sink = glueContext.write_dynamic_frame.from_options(frame=final_frame,
                                                              connection_type="s3",
                                                              connection_options={"path": s3_target},
                                                              format="csv",
                                                              transformation_ctx="s3sink")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
dynaFrame = glueContext.create_dynamic_frame.from_catalog(database="glueworkshop-cloudformation", 
                                                          table_name="json-static-table")
processBatch(dynaFrame.toDF(), "12")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…